# Trabalho-3
## Aluno: Natanael Luciano

caso não tenha o scamp instalado, execute as linha de codigo abaixo no seu cmd:

pip install scamp
pip3 install --user scamp_extensions

a visualização da partitura é feita usando lilypond

Site do scamp: http://scamp.marcevanstein.com/index.html

In [8]:
import numpy as np
import IPython.display as ipd
from ipywidgets import interactive_output #http://ipywidgets.readthedocs.io/en/latest/index.html
from ipywidgets import IntSlider, FloatSlider, fixed, Checkbox
from ipywidgets import VBox, Label
import random
import scipy.stats as st
import librosa, librosa.display  

In [9]:
import random
import scipy.stats as st
from sklearn.utils import resample

<p>
O Objetivo deste trabalho é implementar alguns exemplos de construções músicais interativas fundamentada em uma estrutura de jogo. Ou seja, um jogo musical.
</p>
<p>
Como problema introdutório estaremos recriando um metodo de construção musical apresentado em aula. O método consiste em separar a obra musical nos seus compassos e rearranjar estes conforme o lançamento de um dado de tamanho $n$, onde $n$ é o número de compassos na música.    
</p>
<p>
    por motivos de conveniência, estou supondo que cada compasso equivale à $4$ segundos da obra.
</p>

In [10]:
def divide(music,duration=60):
    #esta função quebra a música em intervalos de tamanho 4*fs, 4 segundos. já que fs é a frequencia de amostragem.
    y, fs = librosa.load(music,None, mono=True, duration = duration)
    z= []
    for i in range(0,len(y),4*fs):
        try:
            z.append(y[i:i + 4*fs])
        except:
            z.append(y[i:])
            return [np.array(z),fs]
    return [np.array(z),fs]

In [11]:
def dice_game(music,duration=60, replace = False):
    #esta função reorganiza os compassos conforme os valores de um dado 
    w_d,fs = divide(music, duration = duration)
    z=[]
    a = resample(w_d,replace = replace, n_samples = len(w_d))    
    for i in a:
        z = z + list(i)
    return [z,fs]

In [12]:
w_d,fs = dice_game('Moonlight.wav', duration = 30)

In [13]:
ipd.Audio(w_d, rate=fs)

<p>
Iremos criar uma variação do primeiro jogo. Agora envolvendo uma moeda e um dado, onde a moeda indica de qual música estaremos retirando o compasso, e o dado qual compasso. 
   </p>

In [117]:
def dice_game_2(musics, duration = 60, replace = False):
    #essa função reorganiza concatena as músicas segundo um dado e uma moeda
    x = []
    for music in musics: 
        w_d,fs = divide(music, duration = duration)
        x.append(w_d)
    z=[]
    for i in range(min(len(x[0]),len(x[1]))):
        a =  random.choice(x)  #escolhendo a música
        aux = resample(a,replace = replace, n_samples = 1)
        for j in aux:
            z = z + list(j)
        #print(len(z))
    return [z,fs]

In [118]:
w_d,fs = dice_game_2(['Moonlight.wav','Guns.wav'], duration = 30)

In [119]:
ipd.Audio(w_d, rate=fs)

Vamos tentar criar um jogo onde o objetivo de um jogardor é criar uma composição sonoramente agradável e o objetivo do segundo jogador é atrapalhar o primeiro o máximo possível. Para isso usaremos cadeias de markov para determinar as escolhas das 'melhores' e piores notas a serem tocadas.

In [121]:
from scamp import *


In [122]:
def Markov_matrix(size, func = st.uniform.rvs):
    #gera a matriz de transição com tamanho size*size, e com valores distribuidos segundo func
    A = func(size = size**2)
    A = A.reshape(size,size)
    for i in range(size):
        A[i][:] = A[i][:]/sum(A[i][:]) #normalizando as probabilidades
    A = np.matrix(A)
    #print('Matriz de transição = {}'.format(A))
    return A

In [204]:
def note_gen(A,pitchs,n_notas,old):
    #função geradora de notas. gera as notas conforme uma cadeia de markov dada pela matriz A
    #gera  a = n_notas notas  
    u = np.zeros(len(pitchs))
    if old[0] == -1:
        z = random.randint(0, len(pitchs) -1 )
        u[z] = 1
        nota = [pitchs[z]]
    else: 
        u[(pitchs.index(old[0]))] = 1
        nota = [old[0]]

    u = u.reshape(1,len(pitchs))
    for i in range(1,n_notas+1):
        vec =  u*(A**i) #vetor das probabilidades
        nota = nota +  list(random.choices(pitchs,weights = vec.T))
        
    if old[0] != -1:
        return nota[1:]
        
    return nota

In [124]:
def janken():
    # simula um jogo de pedra_papel_tesoura e retorna um vetor one_hot dizendo quem ganhou
    w = random.choices([0,1,2],k=2) #pedra papel tesoura
    #print(a)
    if w[0] == 2 and w[1] == 0:
        return [0,1]
    
    if w[0] == 0 and w[1] == 2:
        return [1,0]

    
    if w[0] - w[1] <0:
        return [0,1]
    
    if w[0] == w[1]:
        return[0,0]
    
    else:
        return [1,0]
    

In [276]:
#notas do jogador 1
#"major"
pitchs = list(np.array([24,26,28,29,31,33,35,36]) + 12)
#"quintas"
pitchs_5 = [43,45,47,48,50,52,53,55]
#"oitavas"
pitchs_8 = list(np.array([24,26,28,29,31,33,35,36]) + 24)

notas_1 = pitchs + pitchs_5 + pitchs_8

#notas jogador 2



notas_2 = list(np.array([24,26,28,29,31,33,35,36])+ -3) + \
            list(np.array( [43,45,47,48,50,52,53,55]) + 5) \
            + list(np.array([24,26,28,29,31,33,35,36]) + -4)



In [201]:
all_notas = notas_1 + notas_2

In [219]:
def create(n_rounds = 10):
    M1 = Markov_matrix(len(notas_1))
    M2 = Markov_matrix(len(notas_2))
    n_notas = 1
    notas = []
    dura = list((st.uniform.rvs(loc = 0, scale = 2, size = 1)))
    old1= [-1]
    old2= [-1]
    
    for i in range(n_rounds):
        a = janken()
        while a[0] == a[1]:
            n_notas = n_notas +1
            a = janken()
                
                
                
        if a[0] == 1:
            notas = notas + note_gen(M1,notas_1,n_notas,old1)
            old1[-1] = notas[-1]
            dura= dura +(list(st.uniform.rvs(loc = 0, scale = 2,size = n_notas)))
            #print(n_notas)
            n_notas = 1
        elif a[1] == 1:
            notas = notas +(note_gen(M2,notas_2,n_notas,old2))
            old2[-1] = notas[-1]
            dura = dura +(list(st.uniform.rvs(loc = 0, scale = 2, size = n_notas)))
            #print(n_notas)
            n_notas = 1
    dura.append(list(st.uniform.rvs(loc = 0, scale = 2, size = n_notas)))    
    return[notas,dura]
            
    

In [269]:
a,b = create(n_rounds = 15)
s = Session()
cello = s.new_part('Cello')
s.start_transcribing()
for i ,j in zip(a,b):
    cello.play_note(i,1,j)
s.stop_transcribing().to_score(title = 'Cello Game',composer = 'Natanael Luciano').show_xml()  
#se o senhor quiser ver apenas a partitura substitua por .show()

Using preset Cello LP for Cello


Performance([
   PerformancePart(name='Cello', instrument_id=('Cello', 0), voices={
      '_unspecified_': [
         PerformanceNote(start_beat=0.0, length=1.0595700179773813, pitch=45, volume=1, properties={}),
         PerformanceNote(start_beat=1.0595700179773813, length=0.8837641495493753, pitch=48, volume=1, properties={}),
         PerformanceNote(start_beat=1.9433341675267566, length=0.6205929197556688, pitch=52, volume=1, properties={}),
         PerformanceNote(start_beat=2.5639270872824254, length=1.358577147260088, pitch=56, volume=1, properties={}),
         PerformanceNote(start_beat=3.9225042345425134, length=0.827323454317793, pitch=44, volume=1, properties={}),
         PerformanceNote(start_beat=4.749827688860306, length=0.023048002326327754, pitch=53, volume=1, properties={}),
         PerformanceNote(start_beat=4.772875691186634, length=1.553672273034132, pitch=48, volume=1, properties={}),
         PerformanceNote(start_beat=6.326547964220766, length=0.471262375282

<p>
podemos observar que essa construção musical apresenta notas sem muita harmonia. Entretanto, se observamos bem notamos que as matrizes da cadeia de markov precisam ser matrizes de blocos da forma 
    $$\begin{pmatrix}
A & 0 \\
B & 0 \\
\end{pmatrix}\\$$ para o jogador 1, e 
$$    
\begin{pmatrix}
0 & C \\
0 & D \\
\end{pmatrix}\\
$$
para o jogador 2.
    
Para podermos escolher a melhor nota possível dada uma nota pertencente a $notas_2$ e a pior possível dada uma nota pertencente a $notas_1$.
Já que da maneira atual os jogadores não tem informação das notas escolhidas pelos seus adiversários.
</p>

In [161]:
from scipy.linalg import block_diag

In [353]:
def note_gen_2(A,pitchs,n_notas,old, tps = 'player1'):
    #função geradora de notas. gera as notas conforme uma cadeia de markov dada pela matriz A
    #gera  a = n_notas notas  
    u = np.zeros(len(pitchs))
    if old[0] == -1:
        if tps == 'player1':
            z = random.randint(0,len(notas_1) - 1)
        else:
            z = random.randint(len(notas_1), len(pitchs) -1 )


        u[z] = 1
        nota = [pitchs[z]]
    else: 
        u[(pitchs.index(old[0]))] = 1
        nota = [old[0]]

    u = u.reshape(1,len(pitchs))
    u = u[0]
    #print(u)
    #print('///////////////')
    #print('///////////////')
    #print(len(u*A))
    for i in range(1,n_notas+1):
        #print('///////////////')
        #print(A**i)
        vec =  u*(A**i) #vetor das probabilidades
        #print(vec[:])
        nota = nota +  list(random.choices(pitchs,weights = vec.T))
        
    if old[0] != -1:
        return nota[1:]
        
    return nota

In [352]:
def create_2(n_rounds = 10):
    M1 = Markov_matrix(len(notas_1))
    M2 = Markov_matrix(len(notas_2))
    A =  np.zeros(M2.A.shape)
    B = np.zeros(M1.A.shape)

    M11 = np.block([
        [M1,                A],
        [M2,                B]
            ])
    M22 = np.block([
        [A,                M2],
        [B,                M1]
            ])

    n_notas = 1
    notas = []
    dura = list((st.uniform.rvs(loc = 0, scale = 2, size = 1)))
    old1= [-1]
    old2= [-1]
    
    for i in range(n_rounds):
        a = janken()
        while a[0] == a[1]:
            n_notas = n_notas +1
            a = janken()
                
                
                
        if a[0] == 1:
            notas = notas + note_gen_2(M11,all_notas,n_notas,old1, tps = 'player1')
            old1[-1] = notas[-1]
            dura= dura +(list(st.uniform.rvs(loc = 0, scale = 2,size = n_notas)))
            #print(n_notas)
            n_notas = 1
        elif a[1] == 1:
            notas = notas +(note_gen_2(M22,all_notas,n_notas,old2, tps = 'player2'))
            old2[-1] = notas[-1]
            dura = dura +(list(st.uniform.rvs(loc = 0, scale = 2, size = n_notas )))
            #print(n_notas)
            n_notas = 1
    dura.append(list(st.uniform.rvs(loc = 0, scale = 2, size = n_notas)))    
    return[notas,dura]
            
        


In [351]:
a,b = create_2(n_rounds = 15)
s = Session()
cello = s.new_part('cello')
s.start_transcribing()
for i ,j in zip(a,b):
    cello.play_note(i,1,j)
s.stop_transcribing().to_score(title = 'Cello Game',composer = 'Natanael Luciano').show_xml()  
#se o senhor quiser ver apenas a partitura substitua por .show()

Using preset Cello LP for cello


<p>O mais interessante é que mesmo incluindo notas 'conflitantes' na composição, acabamos gerando algo com uma sonoridade relativamente boa.
</p>